In [90]:
import numpy as np
import pandas as pd

In [91]:
data = np.array(pd.read_csv("Dataset/Text_Emotion_Data.csv"))
stopwords = np.array(pd.read_csv("Dataset/stopwords.txt", sep=" ", header=None)).flatten()

In [92]:
X = data[:, 0]
y = data[:, 1]

In [93]:
import re

def remove_none_alpha(x):
    regex = re.compile('[^a-zA-Z ]')
    return regex.sub('', x)

In [94]:
stopwords = [remove_none_alpha(w) for w in stopwords]

In [95]:
X_seq = []
for sentence in X:
    words_seq = remove_none_alpha(sentence).lower().split(" ")
    words_seq = [w for w in words_seq if len(w) > 2]
    words_seq = [w for w in words_seq if w not in stopwords]
    X_seq.append(words_seq)

In [96]:
max_len = len(max(X_seq, key=len))

# create a new list of lists with the desired size, padding with empty strings
sequences = [['' for _ in range(max_len)] for _ in range(len(X_seq))]

# copy the elements from the original lists into the new list, padding with empty strings as needed
for i, sequence in enumerate(X_seq):
    sequences[i][:len(sequence)] = sequence

In [97]:
import numpy as np

def one_hot_encode(sequence, word_to_idx):
    num_words = len(word_to_idx)
    encoding = np.zeros((len(sequence), num_words))
    for i, word in enumerate(sequence):
        if word in word_to_idx:
            encoding[i, word_to_idx[word]] = 1
    return encoding

def one_hot_encode2(sequence, word_to_idx, max_bit_num):
    encoding = np.zeros((len(sequence), max_bit_num))
    for i, word in enumerate(sequence):
        if word in word_to_idx:            
            binary_string = bin(word_to_idx[word])[2:]  # exclude the "0b" prefix
            binary_array = np.array([int(digit) for digit in binary_string])
            binary_array = np.pad(binary_array, (max_bit_num - len(binary_array), 0), 'constant')

            encoding[i, :] = binary_array
    return encoding

unique_words = list(set([word for sequence in sequences for word in sequence]))
word_to_idx = {word: idx for idx, word in enumerate(unique_words)}

max_bit_num = 1
while(True):
    if(2**max_bit_num > len(unique_words)): break
    max_bit_num += 1

encoded_sequences = []
for sequence in sequences:
    encoded_sequences.append(one_hot_encode2(sequence, word_to_idx, max_bit_num))

encoded_sequences = np.array(encoded_sequences)

In [98]:
X_train = encoded_sequences[:-150]
X_test = encoded_sequences[-150:]
# y_train_str = y[:-150]
# y_test_str = y[-150:]

In [99]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
y_encoded = le.fit_transform(y)

In [100]:
# Define the function you want to apply
def to_binary(num):
    binary_string = bin(num)[2:]  # exclude the "0b" prefix
    binary_array = np.array([int(digit) for digit in binary_string])
    binary_array = np.pad(binary_array, (3 - len(binary_array), 0), 'constant')
    return binary_array

# Create a NumPy array
arr = np.array(y_encoded)

# Initialize the result array
y_encoded_onehot = np.empty((arr.shape[0], 3), dtype=int)

# Apply the function on all elements of the array
for i, num in enumerate(arr):
    y_encoded_onehot[i] = to_binary(num)


In [101]:
y_train = y_encoded_onehot[:-150]
y_test = y_encoded_onehot[-150:]

In [169]:
X_test.shape

(150, 19, 13)

In [192]:
import tensorflow as tf

# Define the Elman network architecture
input_dim = X_train.shape[2]
hidden_dim = 300
output_dim = y_train.shape[1]
num_steps = X_train.shape[1]

inputs = tf.keras.layers.Input(shape=(num_steps, input_dim))
hidden_layer = tf.keras.layers.SimpleRNN(hidden_dim, activation='tanh', return_sequences=True)(inputs)

# Connect the hidden layer to itself at the previous time step
hidden_layer_with_context = tf.keras.layers.SimpleRNN(hidden_dim, activation='tanh', return_sequences=True)(hidden_layer)
hidden_layer_with_context = tf.keras.layers.Concatenate()([hidden_layer, hidden_layer_with_context])

# Flatten the output of the hidden layer
flatten_layer = tf.keras.layers.Flatten()(hidden_layer_with_context)

outputs = tf.keras.layers.Dense(output_dim, activation='softmax')(flatten_layer)

# Define the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
optimizer = tf.keras.optimizers.Adadelta(learning_rate=0.1)
model.compile(optimizer=optimizer, loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)


Epoch 1/100
97/97 [==============================] - 6s 30ms/step - loss: 0.2270
Epoch 2/100
97/97 [==============================] - 3s 30ms/step - loss: 0.2164
Epoch 3/100
97/97 [==============================] - 3s 30ms/step - loss: 0.2128
Epoch 4/100
97/97 [==============================] - 3s 30ms/step - loss: 0.2097
Epoch 5/100
97/97 [==============================] - 3s 30ms/step - loss: 0.2074
Epoch 6/100
97/97 [==============================] - 3s 29ms/step - loss: 0.2042
Epoch 7/100
97/97 [==============================] - 3s 31ms/step - loss: 0.2034
Epoch 8/100
97/97 [==============================] - 3s 29ms/step - loss: 0.1999
Epoch 9/100
97/97 [==============================] - 3s 31ms/step - loss: 0.1968
Epoch 10/100
97/97 [==============================] - 3s 30ms/step - loss: 0.1939
Epoch 11/100
97/97 [==============================] - 3s 30ms/step - loss: 0.1912
Epoch 12/100
97/97 [==============================] - 3s 30ms/step - loss: 0.1886
Epoch 13/100
97/97 [=====

In [ ]:
import tensorflow as tf

# Define the Elman network architecture
input_dim = X_train.shape[2]
hidden_dim = 500
output_dim = y_train.shape[1]
num_steps = X_train.shape[1]

inputs = tf.keras.layers.Input(shape=(num_steps, input_dim))
hidden_layer = tf.keras.layers.SimpleRNN(hidden_dim, activation='relu')(inputs)
outputs = tf.keras.layers.Dense(output_dim, activation='relu')(hidden_layer)

# Define the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)


# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
optimizer = tf.keras.optimizers.Adadelta(learning_rate=0.02)
model.compile(optimizer=optimizer, loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=500, batch_size=32)


In [193]:
import numpy as np

# Evaluate the model on the test data
loss = model.evaluate(X_test, y_test, verbose=0)

# Get the model's predictions on the test data
predictions = model.predict(X_test, verbose=0)

# Convert predictions and labels to class indices
# predicted_labels = np.argmax(predictions, axis=1)
predicted_labels = np.round(predictions)

# Compute accuracy
accuracy = np.mean(predicted_labels == y_test)

print("Accuracy:", np.round(accuracy * 100, 2))


Accuracy: 58.44


In [195]:
np.unique(predicted_labels, axis=0)

array([[0., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)

In [184]:
np.unique(y_test, axis=0)

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0]])